<a href="https://colab.research.google.com/github/OpetherMB/encrypt-decrypt-drive-local/blob/master/Enc2Dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-docx
!pip install cryptography

     |████████████████████████████████| 5.5MB 9.7MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=c65ec5ab1264cd4362df08486b8c47b17aca8289e24e83d05261a9e2d4a063a1
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx
     |████████████████████████████████| 2.6MB 10.3MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import os
import getpass
import docx
from cryptography.fernet import Fernet

import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

Mounted at /content/drive


In [3]:
# # change type from docx to txt
# input_file = '/content/drive/My Drive/temp'

# for File in os.listdir(input_file): 
#   if File.endswith('.docx'):
#       doc = docx.Document(os.path.join(input_file,File) )
#       result = [p.text for p in doc.paragraphs]

#       with open(os.path.join(input_file , File.split('.docx')[0]+".txt") , 'w') as f:
#             f.writelines("%s\n" % line for line in result)
#             f.close()

# add gdoc reader 

In [4]:
def send_mail(sender_email, receiver_email, subject, password , output_file ):
    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message["Bcc"] = receiver_email  # Recommended for mass emails

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    filename = output_file  

    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)

In [5]:
def Encryption_(destination_file, input_folder , key):

      for File in os.listdir(input_folder):

          input_file = os.path.join(input_folder, File )
          output_file = os.path.join(destination_file, File.split('.txt')[0]+'.encrypted')

          with open(input_file, 'rb') as f:
              data = f.read()

          fernet = Fernet(key)
          encrypted = fernet.encrypt(data)

          with open(output_file, 'wb') as f:
              f.write(encrypted)

In [6]:
# You can delete input_file if you want
# And then to decrypt a file:

def TreatFile(destination_file, File, input_folder, key):
         input_file =  os.path.join(destination_file, File )
         outp_ext = File.split('.encrypted')[0]+'.txt'
         output_file =  os.path.join(input_folder, outp_ext )

         with open(input_file, 'rb') as f:
                  data = f.read()

         fernet = Fernet(key)
         encrypted = fernet.decrypt(data)

         with open(output_file, 'wb') as f:
                  f.write(encrypted)

         return output_file                  


def Decryption_(destination_file, input_folder, FileWanted, password ,key , flag ,send_mail ):
      
      if flag : 
              
              output_file = TreatFile(destination_file, FileWanted, input_folder, key)
              send_mail(sender_email, receiver_email, subject, password , output_file)
        


      else :   
          for File in os.listdir(destination_file):
              
              output_file = TreatFile(destination_file, File, input_folder, key)
              send_mail(sender_email, receiver_email, subject, password , output_file)


In [10]:
########
input_folder = "/content/drive/My Drive/temp"
destination_file  ="/content/drive/My Drive/text_files"

###email data 
body = "This is an email with attachment sent from Python"
sender_email = "maroubenm225@gmail.com"
receiver_email = "maroubenm@outlook.com"

### args

send_email = True
encrypt = False
decrypt = True 
FileWanted = 'Lc..ban.encrypted'

subject = "crypted data from drive to you"+FileWanted

#########

if send_email:
  try: 
      print("enter email app code ! ")
      password  = getpass.getpass() 
  except Exception as error: 
      print('ERROR', error) 


try: 
    print("enter the key for encryption")
    key = getpass.getpass()
    key = bytes(key, encoding='utf-8')

except Exception as error:
    print('ERROR', error)


if encrypt : 
    Encryption_(destination_file, input_folder , key)


if decrypt : 
    Decryption_(destination_file, input_folder,FileWanted , password, key, True ,send_mail) 

key = " "
password = " "

# delete all data 
[ os.remove(os.path.join(input_folder , File) ) for File in os.listdir(input_folder) ]

enter email app code ! 
··········
enter the key for encryption
··········


[None]